# Abstractive Text Summarization Using RNN

In [ ]:
print("hello")

hello


## Module Imports

Step 1

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from matplotlib import pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import glob
import itertools
import pickle
import re
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
!pip install rouge-score

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8a8f0925ad0cf49b17561c2df8127b8d0a8b27ceaeeaa9664a50aa685e78b832
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Read Data and Data Preprocessing

Step 2

In [ ]:
train_data=pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')
test_data=pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')
val_data=pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')

In [ ]:
# Splitting train data
X_Train = train_data['article'].values[:100000]
Y_Train = train_data['highlights'].values[:100000]

# Splitting test data
X_Test = test_data['article'].values
Y_Test = test_data['highlights'].values

# Splitting validation data
X_Val = val_data['article'].values
Y_Val = val_data['highlights'].values

STep 3

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import pickle
import itertools
import re
from tqdm import tqdm

Step 4

In [ ]:
drive_prefix = '/kaggle/working/'  # Kaggle's working directory for saving outputs

# STOP_WORDS = set(stopwords.words('english'))
STOP_WORDS = set()
EMB_SIZE = 300
GLOVE_EMB = '/kaggle/input/glove6b300dtxt/glove.6B.300d.txt'
BATCH_SIZE = 32
EPSILON = 0.5 # for deciding between feeding (model's output OR target) as input

START_CHAR = 'starttoken'
END_CHAR = 'endtoken'
PAD_CHAR = 'padtoken'
FORCE_CREATE_DICT = True
MAX_ARTICLE_LEN = 300 # the article can have at most 300 tokens
MAX_DECODER_OUTPUT = 100 # labels can have at most 100 tokens

# NN Hyper-parameters
E_HIDDEN_DIM = 200
D_HIDDEN_DIM = 200

EP = 4400
PRINT_EVERY_EP = 50
SAVE_MODEL_EVERY_EP = 500
FORCE_CREATE_DICT = True # force to recreate the word features from scratch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

# Create model directory if it doesn’t exist
def save_checkpoint(ep, e_model, d_model, e_optim, d_optim, loss, model_name):
    model_dir = f'{drive_prefix}models/{model_name}/'
    os.makedirs(model_dir, exist_ok=True)  # Ensure directory exists

    torch.save({
        'epoch': ep,
        'encoder_model': e_model.state_dict(),
        'decoder_model': d_model.state_dict(),
        'encoder_optimizer': e_optim.state_dict(),
        'decoder_optimizer': d_optim.state_dict(),
        'loss': loss
    }, os.path.join(model_dir, f'checkpoint_{ep}'))
    print(f'Checkpoint saved at epoch {ep}')


cuda


step 5


In [ ]:
def clean_text(s):
    s = s.lower().strip()
    s = s.replace('<t>', '').replace('</t>', '').replace('\n', '')
    s = s.replace('-lrb-', '').replace('-rrb-', '')
    s = re.sub(r'\([^)]*\)', '', s)
    s = re.sub('"','', s)
    s = re.sub(r"'s\b","",s)
    s = re.sub("[^a-zA-Z]", " ", s)
    # s = re.sub(r"([.!?])", r" \1", s)
    # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def create_word_vec(input_text, input_label, force_create):
    word_idx_dict = {} # { word : the index of that word in the dictionary}
    idx_word_dict = {} # { index of word : word }

    # 1. Create mapping between words and the corresponding embedding values
    embed_file_path = drive_prefix + f'{EMB_SIZE}d_embed_dict'
    if os.path.exists(embed_file_path) and not force_create:
        print('Embedding dictionary exists, loading from file...')
        embedding_dict = pickle.load(open(embed_file_path, 'rb'))
    else:
        embedding_dict = {}

        for line in glove:
            tokens = line.split()
            embedding_dict[tokens[0]] = np.array(tokens[1:], dtype='float32')
        pickle.dump(embedding_dict, open(embed_file_path, 'wb'))  # Saving to `/kaggle/working`
        print('Saved embedding dictionary')
    # 2. Tokenize the input_text and labels
    if os.path.exists(drive_prefix + 'train_word_idx_dict') and not force_create:
        print('Word-to-index dictionary exists, loading from file...')
        word_idx_dict = pickle.load(open(drive_prefix + 'train_word_idx_dict', 'rb'))
    if os.path.exists(drive_prefix + 'train_idx_word_dict') and not force_create:
        print('Index-to-word dictionary exists, loading from file...')
        idx_word_dict = pickle.load(open(drive_prefix + 'train_idx_word_dict', 'rb'))
    else:
        unique_tokens = set([])
        for line in input_text:
            unique_tokens = unique_tokens.union(word_tokenize(line))
        for line in input_label:
            unique_tokens = unique_tokens.union(word_tokenize(line))

        for token in unique_tokens:
            word_idx_dict[token] = len(word_idx_dict)

        # 2.1 Add in the special tokens to the dictionary, note that the START_CHAR and END_CHAR have been added
        # during the preprocessing stage
        word_idx_dict[PAD_CHAR] = len(word_idx_dict)

        idx_word_dict = dict(zip(word_idx_dict.values(), word_idx_dict.keys()))

    # 3. Build the word vector for all the words in our dictionary
    if os.path.exists(drive_prefix + 'train_word_vector') and not force_create:
        print('Word Vector exists, loading from file...')
        word_vector = pickle.load(open(drive_prefix + 'train_word_vector', 'rb'))
    else:
        word_vector = []
        for idx, token in idx_word_dict.items():
            if token in embedding_dict:
                word_vector.append(embedding_dict[token])
            # Append the special tokens to the word vector and assign random values
            elif token in [START_CHAR, END_CHAR, PAD_CHAR]:
                word_vector.append(np.random.normal(0, 1, EMB_SIZE))
            # if the token doesn't have an embedding, we set to 0
            else:
                word_vector.append(np.zeros([EMB_SIZE]))

    ## Save the dictionaries
    pickle.dump(word_idx_dict, open(drive_prefix + 'train_word_idx_dict', 'wb'))
    pickle.dump(idx_word_dict, open(drive_prefix + 'train_idx_word_dict', 'wb'))
    pickle.dump(word_vector, open(drive_prefix + 'train_word_vector', 'wb'))

    # The index in word_vec corresponds to the article index in the original X_Test array
    return np.array(word_vector), word_idx_dict, idx_word_dict

def sentence_to_idx(sentence, word_to_idx):
    tokens = word_tokenize(sentence)
    return [word_to_idx[token] for token in tokens if token in word_to_idx]

def decontracted(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return text

def pre_process(column, max_len):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentence in tqdm(column.astype(str)):
        sent = decontracted(sentence)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('\\n', ' ')
        sent = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", sent)
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        sent=re.sub('<[^>]*>', '', sent)
        # trim longer articles
        sent = sent.lower().strip()
        sent = ' '.join(sent.split()[:max_len])
        preprocessed_text.append(sent)
    return preprocessed_text

## TODO: REFERENCE
def zeroPadding(l, fillvalue=PAD_CHAR):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_CHAR):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_CHAR:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def plot(train_loss, val_loss):
    plt.plot(train_loss, label='Train')
    plt.plot(val_loss, label='Val')
    plt.title('Training vs Validation Loss')
    plt.xlabel('Episodes')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

Step 6

In [ ]:
with open(GLOVE_EMB, 'r', encoding='utf-8') as f:
    glove = f.readlines()

Step 7

In [ ]:
#Train and Validation split
X_Test = pre_process(np.array(X_Test), max_len=MAX_ARTICLE_LEN)
Y_Test = pre_process(np.array(Y_Test), max_len=MAX_DECODER_OUTPUT)
Y_Test = list(map(lambda x: f"{START_CHAR} {x} {END_CHAR}", Y_Test))

X_Test, X_Val, Y_Test, Y_Val = train_test_split(X_Test, Y_Test, test_size=0.3, random_state=23, shuffle=True)
print(f'\nTotal # of stories: {len(X_Test)}')

100%|██████████| 11490/11490 [00:01<00:00, 8467.67it/s]


Total # of stories: 8043


In [ ]:
max(list(map(len, X_Test)))

2025

## Features

Step 8

In [ ]:
train_word_vector, train_word_idx_dict, train_idx_word_dict = create_word_vec(X_Test, Y_Test, force_create=FORCE_CREATE_DICT)
print(f'Word Vector Shape: {train_word_vector.shape}')
assert train_word_vector.shape == (len(train_idx_word_dict.keys()), EMB_SIZE)

Saved embedding dictionary
Word Vector Shape: (68168, 300)


In [ ]:
# Create the embedding layer weights based on the pre-trained word vector
def create_pretrained_emb_layer(word_vector):
    # vocab_size, embed_dim = word_vector.shape
    embed_layer = nn.Embedding.from_pretrained(torch.tensor(word_vector).float(), freeze=False)
    return embed_layer

STep 9

In [ ]:
class Encoder(nn.Module):
    def __init__(self, hidden_dim, embedding_layer, n_layers=1, dropout=0):
        super(Encoder, self).__init__()

        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = embedding_layer
        self.gru = nn.GRU(EMB_SIZE,
                          hidden_dim,
                          num_layers=n_layers,
                          dropout=dropout,
                          bidirectional=True)

    def forward(self, x, lengths, x_h=None):
        embedded = self.embedding(x)
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, lengths, enforce_sorted=False)
        # Forward pass through GRU
        y_hat, x_h = self.gru(packed, x_h)
        # Unpack padding
        y_hat, _ = torch.nn.utils.rnn.pad_packed_sequence(y_hat)
        # Sum bidirectional GRU outputs
        y_hat = y_hat[:, :, :self.hidden_dim] + y_hat[:, : ,self.hidden_dim:]
        # Return output and final hidden state

        return y_hat, x_h

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim

        self.attention = nn.Linear(hidden_dim * 2, hidden_dim)
        self.value = nn.Parameter(torch.FloatTensor(hidden_dim).normal_())

    def forward(self, x_h, encoder_output):
        energy = self.attention(torch.cat((x_h.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        attn_score = torch.sum(self.value * energy, dim=2)

        # Transpose max_length and batch_size dimensions
        attn_score = attn_score.t()

        # the softmax normalized probability scores (with added dimension)
        attn_weights = F.softmax(attn_score, dim=1).unsqueeze(1)
        return attn_weights


class Decoder(nn.Module):
    def __init__(self, hidden_dim, output_dim, embedding_layer, n_layers=1, dropout=0.1):
        super(Decoder, self).__init__()

        self.embedding = embedding_layer # TODO
        self.dropout = nn.Dropout(dropout)
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size=EMB_SIZE,
                          hidden_size=hidden_dim,
                          num_layers=n_layers,
                          dropout=dropout)

        self.attn_gru_combined = nn.Linear(hidden_dim * 2, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

        self.attention = Attention(hidden_dim)

    def forward(self, x, x_h, encoder_output):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)

        y_hat, x_h = self.gru(embedded, x_h)

        attn_weights = self.attention(y_hat, encoder_output)

        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_output.transpose(0, 1))

        # Concatenate weighted context vectors
        y_hat = y_hat.squeeze(0)
        context = context.squeeze(1)
        weighted_context = torch.cat((y_hat, context), 1)

        y_hat = torch.tanh(self.attn_gru_combined(weighted_context))

        y_hat = F.softmax(self.out(y_hat), dim=1)
        return y_hat, x_h

Step 10

In [ ]:
def maskNLLLoss(inp, target, mask):
    # TODO: Reference
    mask = mask.bool()
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

def one_pass(x, y, e_model, d_model, e_optim, d_optim, lengths, mask, max_target_len):
    e_optim.zero_grad()
    d_optim.zero_grad()

    x = torch.tensor(x).to(device)
    y = torch.tensor(y).to(device)
    mask = mask.to(device)

    loss = 0 # loss accumulated from each timestep
    pass_loss = [] # loss for one pass

    e_output, e_hidden = e_model(x, lengths)

    # Create initial decoder input (start with START_CHAR for each sequence)
    d_input = torch.LongTensor([[train_word_idx_dict[START_CHAR] for _ in range(BATCH_SIZE)]])
    d_input = d_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    d_hidden = e_hidden[:d_model.n_layers]

    for i in range(max_target_len):
        d_output, d_hidden = d_model(d_input, d_hidden, e_output)

        if random.random() < EPSILON:
            d_input = y[i].view(1, -1)

        else:
            # TODO
            _, topi = d_output.topk(1) # topi = torch.argmax(d_output, axis=1)
            d_input = torch.LongTensor([[topi[i][0] for i in range(BATCH_SIZE)]])
            d_input = d_input.to(device)

        # Calculate and accumulate loss
        mask_loss, nTotal = maskNLLLoss(d_output, y[i], mask[i])
        loss += mask_loss

        pass_loss.append(mask_loss.item())


    loss.backward()
    torch.nn.utils.clip_grad_norm_(e_model.parameters(), 1)
    torch.nn.utils.clip_grad_norm_(d_model.parameters(), 1)

    e_optim.step()
    d_optim.step()

    return sum(pass_loss) / len(pass_loss)

import os
import glob
import torch

def load_checkpoint_if_available(e_model, d_model, e_optim, d_optim, model_name):
    # Check for existing checkpoints
    checkpoint_files = glob.glob(f'/kaggle/working/models/{model_name}/checkpoint_*.pth')
    if checkpoint_files:
        # Load the latest checkpoint based on creation time
        latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
        print(f"Loading checkpoint: {latest_checkpoint}")
        checkpoint = torch.load(latest_checkpoint)

        # Load model and optimizer states
        e_model.load_state_dict(checkpoint['encoder_model'])
        d_model.load_state_dict(checkpoint['decoder_model'])
        e_optim.load_state_dict(checkpoint['encoder_optimizer'])
        d_optim.load_state_dict(checkpoint['decoder_optimizer'])
        start_epoch = checkpoint['epoch'] + 1  # Start from the next epoch
        print(f"Resuming from epoch {start_epoch}")
        return start_epoch
    else:
        print("No checkpoint found. Starting training from scratch.")
        return 1  # Start from the first epoch if no checkpoint is found

def train(e_model, d_model, e_optim, d_optim, model_name):
    print('Training')
    e_model.train()
    d_model.train()
    ep_loss = []

    # Define and create the directory for saving models
    save_dir = f'/kaggle/working/models/{model_name}'
    os.makedirs(save_dir, exist_ok=True)
    print("Model save directory:", save_dir)

    # Load from checkpoint if available
    start_epoch = load_checkpoint_if_available(e_model, d_model, e_optim, d_optim, model_name)

    for ep in range(start_epoch, EP + 1):  # Start from loaded epoch if applicable
        X_samples = []
        Y_samples = []
        for _ in range(BATCH_SIZE):
            rand_idx = random.randint(0, len(X_Test) - 1)
            X_samples.append(X_Test[rand_idx])
            Y_samples.append(Y_Test[rand_idx])

        # Process input data
        indexes_batch = [sentence_to_idx(sentence, train_word_idx_dict) for sentence in X_samples]
        lengths = torch.tensor([len(indexes) for indexes in indexes_batch], dtype=torch.int64).cpu()
        padList = zeroPadding(indexes_batch, fillvalue=train_word_idx_dict[PAD_CHAR])
        X_batch = torch.LongTensor(padList).to(device)

        # Process labels
        indexes_batch = [sentence_to_idx(sentence, train_word_idx_dict) for sentence in Y_samples]
        max_target_len = max([len(indexes) for indexes in indexes_batch])
        padList = zeroPadding(indexes_batch, fillvalue=train_word_idx_dict[PAD_CHAR])
        mask = torch.BoolTensor(binaryMatrix(padList)).to(device)
        Y_batch = torch.LongTensor(padList).to(device)

        # Run one training pass
        loss = one_pass(X_batch, Y_batch, e_model, d_model, e_optim, d_optim, lengths, mask, max_target_len)
        ep_loss.append(loss)

        # Print loss every PRINT_EVERY_EP epochs
        if ep % PRINT_EVERY_EP == 0 and ep > 0:
            print(f'EP:{ep} | Loss: {np.array(ep_loss[-PRINT_EVERY_EP:]).mean():.4f}')

        # Save checkpoint at every SAVE_MODEL_EVERY_EP epochs
        if ep % SAVE_MODEL_EVERY_EP == 0 and ep > 0:
            checkpoint_path = f'{save_dir}/checkpoint_{ep}.pth'
            print(f"Saving checkpoint at epoch {ep} to {checkpoint_path}...")
            torch.save({
                'epoch': ep,
                'encoder_model': e_model.state_dict(),
                'decoder_model': d_model.state_dict(),
                'encoder_optimizer': e_optim.state_dict(),
                'decoder_optimizer': d_optim.state_dict(),
                'loss': loss
            }, checkpoint_path)

    # Save the final model
    final_model_path = f'{save_dir}/final_model.pth'
    print(f"Saving final model to {final_model_path}...")
    torch.save({
        'epoch': EP,
        'encoder_model': e_model.state_dict(),
        'decoder_model': d_model.state_dict(),
        'encoder_optimizer': e_optim.state_dict(),
        'decoder_optimizer': d_optim.state_dict(),
        'loss': loss
    }, final_model_path)



step 11

In [ ]:
import os
import nltk

# Set the directory for NLTK data
nltk_data_dir = '/kaggle/working/nltk_data'
os.makedirs(nltk_data_dir, exist_ok=True)

# Set the NLTK_DATA environment variable to point to the new directory
os.environ['NLTK_DATA'] = nltk_data_dir

# Download the required NLTK packages
nltk.download('wordnet', download_dir=nltk_data_dir)
nltk.download('omw-1.4', download_dir=nltk_data_dir)  # Optional for better word matching
nltk.download('punkt', download_dir=nltk_data_dir)

# Check if the downloads are successful by verifying the corpus location
print("NLTK data directory set to:", nltk_data_dir)


[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package punkt to /kaggle/working/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
NLTK data directory set to: /kaggle/working/nltk_data


Step 12

In [ ]:
# # MAIN DRIVER
model_name = 'autoencoder_attn'

e_embedding_layer = create_pretrained_emb_layer(train_word_vector)
d_embedding_layer = create_pretrained_emb_layer(train_word_vector)

assert train_word_vector.shape[0] == len(train_word_idx_dict) == len(train_idx_word_dict)

encoder = Encoder(E_HIDDEN_DIM, embedding_layer=e_embedding_layer, n_layers=3, dropout=0.3).to(device)
decoder = Decoder(D_HIDDEN_DIM, embedding_layer=d_embedding_layer, output_dim=train_word_vector.shape[0], n_layers=3).to(device)

e_optim = optim.Adam(encoder.parameters(), lr=1e-3)
d_optim = optim.Adam(decoder.parameters(), lr=1e-3)

# Load the latest checkpoint if available
save_dir = '/kaggle/working/models/autoencoder_attn'
checkpoint_files = glob.glob(f'{save_dir}/checkpoint_*.pth')
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    print(f"Loading checkpoint: {latest_checkpoint}")
    checkpoint = torch.load(latest_checkpoint)

    # Load model and optimizer states
    encoder.load_state_dict(checkpoint['encoder_model'])
    decoder.load_state_dict(checkpoint['decoder_model'])
    e_optim.load_state_dict(checkpoint['encoder_optimizer'])
    d_optim.load_state_dict(checkpoint['decoder_optimizer'])
    start_epoch = checkpoint['epoch'] + 1  # Start from the next epoch
    print(f"Resuming training from epoch {start_epoch}")

    # Ensure models are on the correct device
    encoder.to(device)
    decoder.to(device)
else:
    start_epoch = 1  # Start from the first epoch if no checkpoint is found
    print("No checkpoint found. Starting training from scratch.")


No checkpoint found. Starting training from scratch.


In [ ]:
plot_loss = train(encoder, decoder, e_optim, d_optim, model_name=model_name)

Training
Model save directory: /kaggle/working/models/autoencoder_attn
No checkpoint found. Starting training from scratch.


/tmp/ipykernel_30/1861935463.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).to(device)
/tmp/ipykernel_30/1861935463.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(device)


EP:50 | Loss: 5.6979
EP:100 | Loss: 4.2565
EP:150 | Loss: 4.3401
EP:200 | Loss: 4.1387
EP:250 | Loss: 4.1766
EP:300 | Loss: 4.1981
EP:350 | Loss: 4.1431
EP:400 | Loss: 4.1599
EP:450 | Loss: 4.1007
EP:500 | Loss: 4.0505
Saving checkpoint at epoch 500 to /kaggle/working/models/autoencoder_attn/checkpoint_500.pth...
EP:550 | Loss: 4.0449
EP:600 | Loss: 4.0140
EP:650 | Loss: 3.9157
EP:700 | Loss: 3.8947
EP:750 | Loss: 4.0075
EP:800 | Loss: 3.9175
EP:850 | Loss: 3.8598
EP:900 | Loss: 3.8715
EP:950 | Loss: 3.8499
EP:1000 | Loss: 3.8241
Saving checkpoint at epoch 1000 to /kaggle/working/models/autoencoder_attn/checkpoint_1000.pth...
EP:1050 | Loss: 3.7509
EP:1100 | Loss: 3.6756
EP:1150 | Loss: 3.8230
EP:1200 | Loss: 3.7330
EP:1250 | Loss: 3.7677
EP:1300 | Loss: 3.6767
EP:1350 | Loss: 3.6415
EP:1400 | Loss: 3.6980
EP:1450 | Loss: 3.5875
EP:1500 | Loss: 3.6447
Saving checkpoint at epoch 1500 to /kaggle/working/models/autoencoder_attn/checkpoint_1500.pth...
EP:1550 | Loss: 3.5836
EP:1600 | Loss:

In [ ]:
# Uninstall the previous installed nltk library
!pip install -U nltk

# This upgraded nltkto version 3.5 in which meteor_score is there.
!pip install nltk==3.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.1 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434678 sha256=7b5c86583faf57d63afbd556826bf0b71d1ec588e4a50697644f1e4f06340f3e
  Stored in directory: /root/.cache/pip/wheels/35/ab/82/f9667f6f884d272670a15382599a9c753a1dfdc83f7412e37d
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.

In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
import os

# Path to check if the WordNet corpus exists
wordnet_path = '/kaggle/working/nltk_data/corpora/wordnet'
if os.path.exists(wordnet_path):
    print("WordNet corpus found in:", wordnet_path)
else:
    print("WordNet corpus not found. Please check the directory structure.")


WordNet corpus found in: /kaggle/working/nltk_data/corpora/wordnet


In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score


In [ ]:
# Step 3: Function to evaluate a single article using the trained encoder and decoder
def evaluate(article, encoder, decoder, word_idx_dict, idx_word_dict, device):
    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        # Convert article to a list of indices
        indexes_batch = [sentence_to_idx(article, word_idx_dict)]
        lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
        input_batch = torch.LongTensor(indexes_batch).transpose(0, 1).to(device)

        # Pass through the encoder
        e_output, e_hidden = encoder(input_batch, lengths)
        d_hidden = e_hidden[:decoder.n_layers]

        # Initialize decoder input with START_CHAR
        d_input = torch.ones(1, 1, device=device, dtype=torch.long) * word_idx_dict[START_CHAR]

        # Store the generated tokens
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)

        for _ in range(MAX_DECODER_OUTPUT):
            d_output, d_hidden = decoder(d_input, d_hidden, e_output)
            scores, d_input = torch.max(d_output, dim=1)
            all_tokens = torch.cat((all_tokens, d_input), dim=0)
            d_input = torch.unsqueeze(d_input, 0)

            # Stop if END_CHAR is generated
            if idx_word_dict[d_input.item()] == END_CHAR:
                break

        # Decode the tokens to words
        decoded_words = [idx_word_dict[token.item()] for token in all_tokens]
        summary = ' '.join(decoded_words).replace(f' {END_CHAR}', '').strip()

    return summary

# Step 4: Function to compute evaluation metrics
def compute_evaluation_metrics(pred_summaries, true_summaries):
    # ROUGE calculation
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    for i in range(len(pred_summaries)):
        scores = scorer.score(pred_summaries[i], true_summaries[i])
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    # METEOR calculation
    meteor_scores = [meteor_score([true_summaries[i]], pred_summaries[i]) for i in range(len(pred_summaries))]

    # BERTScore calculation
    P, R, F1 = bert_score(pred_summaries, true_summaries, lang='en', rescale_with_baseline=False)

    # Return computed metrics
    return {
        'rouge1_fmeasure': rouge1_scores,
        'rouge2_fmeasure': rouge2_scores,
        'rougeL_fmeasure': rougeL_scores,
        'meteor': meteor_scores,
        'bertscore_f1': F1.tolist()  # Convert tensor to list for consistency
    }

# Step 5: Model evaluation setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sample articles and true summaries for evaluation
sample_articles = X_Test[:288]  # Take 1/50 part of test articles
true_summaries = Y_Test[:288]

# Generate predicted summaries
predicted_summaries = [evaluate(article, encoder, decoder, train_word_idx_dict, train_idx_word_dict, device) for article in sample_articles]

# Compute evaluation metrics
metrics = compute_evaluation_metrics(predicted_summaries, true_summaries)

# Print evaluation results in percentage format
print("Evaluation Metrics:")
print(f"Average ROUGE-1 F1: {np.mean(metrics['rouge1_fmeasure']) * 100:.2f}")
print(f"Average ROUGE-2 F1: {np.mean(metrics['rouge2_fmeasure']) * 100:.2f}")
print(f"Average ROUGE-L F1: {np.mean(metrics['rougeL_fmeasure']) * 100:.2f}")
print(f"Average METEOR: {np.mean(metrics['meteor']) * 100:.2f}")
print(f"Average BERTScore F1: {np.mean(metrics['bertscore_f1']) * 100:.2f}")


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation Metrics:
Average ROUGE-1 F1: 14.32
Average ROUGE-2 F1: 2.21
Average ROUGE-L F1: 12.01
Average METEOR: 9.23
Average BERTScore F1: 75.06
